## Import the database from MongoDB

In [1]:
from pymongo import MongoClient
import pandas as pd
from json import loads, dumps

In [2]:
mongo = MongoClient(port=27017)

In [3]:
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'classroomDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'listings_db', 'local', 'met', 'travelDB', 'uk_food']


In [4]:
listings = mongo['listings_db'].nyc_listings
print(listings)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'listings_db'), 'nyc_listings')


In [5]:
listings_arr = listings.find()
listings_df = pd.DataFrame(listings_arr)
listings_df.head()

,_id,id,listing_url,name,neighborhood_overview,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,...,bathrooms_text,beds,amenities,price,number_of_reviews,first_review,last_review,review_scores_rating,reviews_per_month,bedrooms
0,666db48268a7bdc51a367f80,2595,https://www.airbnb.com/rooms/2595,Rental unit in New York · ★4.68 · Studio · 1 b...,Centrally located in the heart of Manhattan ju...,Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,...,1 bath,1.0,"[Long term stays allowed, Cooking basics, Ethe...",$240.00,49,2009-11-21,2022-06-21,4.68,0.29,NaN
1,666db48268a7bdc51a367f81,5121,https://www.airbnb.com/rooms/5121,Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...,NaN,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,...,NaN,1.0,"[Air conditioning, Wifi, Kitchen, Heating]",$66.00,50,2009-05-28,2019-12-02,4.52,0.28,NaN
2,666db48268a7bdc51a367f82,6848,https://www.airbnb.com/rooms/6848,Rental unit in Brooklyn · ★4.58 · 2 bedrooms ·...,NaN,Williamsburg,Brooklyn,40.70935,-73.95342,Entire rental unit,...,1 bath,1.0,"[Cooking basics, Microwave, Extra pillows and ...",$81.00,191,2009-05-25,2023-08-14,4.58,1.08,NaN
3,666db48268a7bdc51a367f83,6990,https://www.airbnb.com/rooms/6990,Rental unit in New York · ★4.88 · 1 bedroom · ...,"Location: Five minutes to Central Park, Museum...",East Harlem,Manhattan,40.78778,-73.94759,Private room in rental unit,...,1 shared bath,1.0,"[Hair dryer, Air conditioning, Fire extinguish...",$70.00,246,2009-10-28,2023-08-14,4.88,1.43,NaN
4,666db48268a7bdc51a367f84,6872,https://www.airbnb.com/rooms/6872,Condo in New York · 1 bedroom · 1 bed · 1 shar...,This sweet Harlem sanctuary is a 10-20 minute ...,East Harlem,Manhattan,40.80107,-73.94255,Private room in condo,...,1 shared bath,1.0,"[Fire extinguisher, Long term stays allowed, H...",$65.00,1,2022-06-05,2022-06-05,5.00,0.05,NaN


## Clean the data

In [6]:
listings_df.dtypes

_id                                     object
id                                       int64
listing_url                             object
name                                    object
neighborhood_overview                   object
neighbourhood_cleansed                  object
neighbourhood_group_cleansed            object
latitude                               float64
longitude                              float64
property_type                           object
room_type                               object
accommodates                             int64
bathrooms_text                          object
beds                                   float64
amenities                               object
price                                   object
number_of_reviews                        int64
first_review                    datetime64[ns]
last_review                     datetime64[ns]
review_scores_rating                   float64
reviews_per_month                      float64
bedrooms     

In [7]:
reduced_listings = listings_df[['name', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'\
                               ,'latitude','longitude','room_type', 'amenities', 'price'\
                                , 'number_of_reviews', 'review_scores_rating']]
reduced_listings.dtypes

name                             object
neighbourhood_cleansed           object
neighbourhood_group_cleansed     object
latitude                        float64
longitude                       float64
room_type                        object
amenities                        object
price                            object
number_of_reviews                 int64
review_scores_rating            float64
dtype: object

In [8]:
reduced_listings['price'] = reduced_listings['price'].str.replace(",", "")
reduced_listings['price'] = reduced_listings['price'].str.replace("$", "")

reduced_listings['price']

/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/95660149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_listings['price'] = reduced_listings['price'].str.replace(",", "")
/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/95660149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_listings['price'] = reduced_listings['price'].str.replace("$", "")


0        240.00
1         66.00
2         81.00
3         70.00
4         65.00
          ...  
28127    225.00
28128    101.00
28129    105.00
28130    118.00
28131    285.00
Name: price, Length: 28132, dtype: object

In [9]:
reduced_listings = reduced_listings.astype({
    "price": float
})

reduced_listings.dtypes

name                             object
neighbourhood_cleansed           object
neighbourhood_group_cleansed     object
latitude                        float64
longitude                       float64
room_type                        object
amenities                        object
price                           float64
number_of_reviews                 int64
review_scores_rating            float64
dtype: object

In [10]:
reduced_listings.to_json("data/cleaned_listings.json", orient="records")


# Aggregate Functions

In [25]:
boroughs = reduced_listings["neighbourhood_group_cleansed"].unique()
for i in boroughs:
    print(i)

Manhattan
Brooklyn
Queens
Staten Island
Bronx


## Filtering main listings DataFrame by Borough:

In [102]:
def borough_listings(borough): 
    borough_clean = reduced_listings[reduced_listings["neighbourhood_group_cleansed"] == borough]
    return borough_clean

for borough in boroughs: 
    listings = borough_listings(borough)


In [113]:
#Defining listing var outside of for loop for rest of agg funcs: 
listings = borough_listings(borough)

## Group by neighborhood, find average price:

In [109]:
def avg_price_nbhd(borough):
    
    avg_price = listings.groupby(["neighbourhood_cleansed"])["price"].mean()
    avg_price_boro = avg_price.round(2).reset_index() 
    avg_price_boro["borough"]= borough
    return avg_price_boro 

for borough in boroughs: 
    print(avg_price_nbhd(borough))

   neighbourhood_cleansed   price    borough
0                Allerton  101.76  Manhattan
1              Baychester   99.20  Manhattan
2                 Belmont  104.35  Manhattan
3               Bronxdale   72.88  Manhattan
4             Castle Hill  123.20  Manhattan
5             City Island  182.31  Manhattan
6       Claremont Village  111.94  Manhattan
7            Clason Point  105.72  Manhattan
8              Co-op City   78.60  Manhattan
9               Concourse   98.29  Manhattan
10      Concourse Village  106.60  Manhattan
11           Country Club  161.50  Manhattan
12        East Morrisania   98.38  Manhattan
13            Eastchester  130.12  Manhattan
14               Edenwald   85.06  Manhattan
15              Fieldston   92.75  Manhattan
16                Fordham   90.41  Manhattan
17             Highbridge   87.06  Manhattan
18            Hunts Point   56.50  Manhattan
19            Kingsbridge   90.30  Manhattan
20               Longwood  367.31  Manhattan
21        

## Room type count per borough neighborhoods: 

In [116]:
def roomtype_cleansed(borough):
    
    listings = borough_listings(borough)
    
    roomtype_grouped = listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')
    roomtype_cleaned = roomtype_grouped.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)
    return roomtype_cleaned
    
for borough in boroughs:
    print(roomtype_cleansed(borough))
    


room_type               Entire home/apt  Hotel room  Private room  Shared room
neighbourhood_cleansed                                                        
Battery Park City                  34.0         0.0           9.0          0.0
Chelsea                           424.0         2.0         122.0          6.0
Chinatown                         128.0         0.0          88.0          1.0
Civic Center                       13.0         0.0           5.0          0.0
East Harlem                       293.0         0.0         240.0         13.0
East Village                      524.0         0.0         246.0          6.0
Financial District                175.0         0.0         172.0          6.0
Flatiron District                  35.0         2.0          21.0          0.0
Gramercy                          138.0         0.0          53.0          1.0
Greenwich Village                 125.0         2.0          22.0          0.0
Harlem                            658.0         1.0 

## Grouping by Room Type and merging with avg price:


In [57]:
def room_type_agg(borough):

    borough_listings = reduced_listings[reduced_listings["neighbourhood_group_cleansed"] == borough].reset_index()
    
    grouped_room_type_num = borough_listings.groupby(['neighbourhood_cleansed', 'room_type']).count()

    grouped_room_type_num = grouped_room_type_num['name']

    grouped_price = borough_listings.groupby(['neighbourhood_cleansed', 'room_type']).mean('price')

    grouped_price = grouped_price[["price"]]

    grouped_room_type = pd.merge(grouped_price, grouped_room_type_num, how='outer', on=['neighbourhood_cleansed', 'room_type'])

    grouped_room_type = grouped_room_type.rename(columns={'price': 'Average Price', 'name': 'Room Type Count'}).reset_index(names=['Neighbourhood', 'Room Type'])

    grouped_room_type.to_json(f"data/grouped_room_type_listings_{borough}.json", orient="records")
    
    return grouped_room_type

for borough in boroughs:
    room_type_agg(borough)


## Specific Amenity counts per borough: 

In [121]:
def amenities_by_nbhd(borough, keywords):
    
    listings = borough_listings(borough)
    keywords = ["Wifi", "Kitchen", "Air conditioning", "Backyard", "Pets allowed", "Washer", "Dryer", "Long term stays allowed", "Elevator", "Dedicated workspace"]
    
    # Convert amenities column to string and split into elements using 'strip':
    
    listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')
    
    # Explode the amenities column:
    
    listings_exploded = listings.explode('amenities').copy()
    
    # Initialize a DataFrame to store the counts
    
    neighborhood_counts = listings['neighbourhood_cleansed'].drop_duplicates().to_frame()
    
    # Count the occurrences of each keyword in the amenities column by neighborhood
    
    for keyword in keywords:
        keyword_count = listings_exploded[listings_exploded['amenities'].str.contains(keyword, case=False)]
        keyword_count = keyword_count.groupby('neighbourhood_cleansed').size().reset_index(name=keyword)
        neighborhood_counts = pd.merge(neighborhood_counts, keyword_count, on='neighbourhood_cleansed', how='left')
        
    # Fill NaN values with 0
    neighborhood_counts = neighborhood_counts.fillna(0)
    neighborhood_counts.to_json(f"data/amenities_counts_nbhd{borough}.json", orient="records")
    return neighborhood_counts
    
for borough in boroughs:
    print(amenities_by_nbhd(borough, keywords))

/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/3522188396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')


   neighbourhood_cleansed  Wifi  Kitchen  Air conditioning  Backyard  \
0                 Midtown  1063      831               898      22.0   
1             East Harlem   550      506               416      53.0   
2          Hell's Kitchen  1042      910               885      56.0   
3            East Village   772      746               610      54.0   
4                 Chelsea   559      507               451      33.0   
5         Lower East Side   473      443               387      17.0   
6         Upper East Side  1040     1009               886      63.0   
7                Kips Bay   263      252               213       9.0   
8                  Harlem  1397     1295               951     158.0   
9            West Village   360      350               290      32.0   
10        Upper West Side   963      899               738      48.0   
11    Morningside Heights   129      129                95       1.0   
12              Chinatown   218      183               161      

/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/3522188396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')


       neighbourhood_cleansed  Wifi  Kitchen  Air conditioning  Backyard  \
0          Bedford-Stuyvesant  2061     2000              1466     419.0   
1                Williamsburg  1773     1708              1387     237.0   
2                 Fort Greene   251      239               181      40.0   
3                  Greenpoint   514      464               422      53.0   
4                Clinton Hill   274      265               216      56.0   
5             Carroll Gardens   129      126               105      25.0   
6                  Park Slope   246      233               183      60.0   
7                 South Slope   123      118                97      30.0   
8               East Flatbush   430      399               259     115.0   
9                 Boerum Hill    91       87                69      17.0   
10           Prospect Heights   162      157               124      27.0   
11           Brooklyn Heights    65       62                43       5.0   
12          

/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/3522188396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')


   neighbourhood_cleansed  Wifi  Kitchen  Air conditioning  Backyard  \
0        Long Island City   327    285.0             255.0      61.0   
1                Woodside   253    235.0              90.0      14.0   
2               Sunnyside   219    201.0             128.0      21.0   
3               Ridgewood   221    210.0             136.0      35.0   
4          Middle Village    23     23.0              15.0       4.0   
5        Ditmars Steinway   160    150.0              90.0      32.0   
6                 Jamaica   258    223.0             157.0     100.0   
7                 Astoria   512    471.0             340.0      39.0   
8         Jackson Heights   170    156.0             104.0      24.0   
9          Rockaway Beach    57     55.0              32.0      20.0   
10              Rego Park    64     58.0              33.0      10.0   
11                Maspeth    79     71.0              48.0      16.0   
12              Briarwood    38     27.0              26.0      

/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/3522188396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')
/var/folders/76/jb4rsfs93334vqf862rsx8jh0000gn/T/ipykernel_4079/3522188396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['amenities'] = listings['amenities'].astype(str).str.strip('[]').str.split(', ')


   neighbourhood_cleansed  Wifi  Kitchen  Air conditioning  Backyard  \
0             Eastchester     8        7                 7       5.0   
1             Kingsbridge    62       53                42       7.0   
2      University Heights     5        6                 3       1.0   
3                Allerton    38       35                21      13.0   
4          Spuyten Duyvil     3        2                 3       2.0   
5              Co-op City     5        4                 3       0.0   
6              Mott Haven    66       58                42      15.0   
7                 Melrose    20       18                10       1.0   
8                 Norwood    27       26                17       1.0   
9               Concourse    39       33                19       3.0   
10            City Island    13       11                 7       8.0   
11            Port Morris    21       17                16       4.0   
12               Longwood    37       36                25      

## Ratings Aggregation: 

In [122]:
def avgRatings(borough): 

    listings = borough_listings(borough)
    ratings_df= listings.groupby(["neighbourhood_cleansed"])["review_scores_rating"].mean()
    return ratings_df

avgrating = avgRatings("Manhattan")
print(avgrating)

neighbourhood_cleansed
Battery Park City      4.818372
Chelsea                4.705487
Chinatown              4.653410
Civic Center           4.865556
East Harlem            4.655275
East Village           4.691959
Financial District     4.679377
Flatiron District      4.610517
Gramercy               4.740573
Greenwich Village      4.720470
Harlem                 4.671516
Hell's Kitchen         4.598905
Inwood                 4.552302
Kips Bay               4.668914
Little Italy           4.694235
Lower East Side        4.659286
Marble Hill            4.917143
Midtown                4.579880
Morningside Heights    4.644737
Murray Hill            4.615263
NoHo                   4.810417
Nolita                 4.767594
Roosevelt Island       4.628929
SoHo                   4.701475
Stuyvesant Town        4.557857
Theater District       4.520340
Tribeca                4.703232
Two Bridges            4.720000
Upper East Side        4.651773
Upper West Side        4.660220
Washington Height

## ToDo Graphing:

bar chart
    stacked amenities by neighbourhood
        (potential drop-down for specified amenities)

pie chart
    percent of each room type for the borough

scatter plot
    average price distribution by room type
        (a scatter plot for each room type with regression line)

box plot
    price by borough
        (the price of all)



# DELETE EVERYTHING BELOW THIS LINE?

## Create DataFrames for each borough

### Manhattan Aggregates

In [13]:
manhattan_listings = reduced_listings[listings_df["neighbourhood_group_cleansed"] == "Manhattan"]

manhattan_listings.head()

,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,amenities,price,number_of_reviews,review_scores_rating
0,Rental unit in New York · ★4.68 · Studio · 1 b...,Midtown,Manhattan,40.75356,-73.98559,Entire home/apt,"[Long term stays allowed, Cooking basics, Ethe...",240.0,49,4.68
3,Rental unit in New York · ★4.88 · 1 bedroom · ...,East Harlem,Manhattan,40.78778,-73.94759,Private room,"[Hair dryer, Air conditioning, Fire extinguish...",70.0,246,4.88
4,Condo in New York · 1 bedroom · 1 bed · 1 shar...,East Harlem,Manhattan,40.80107,-73.94255,Private room,"[Fire extinguisher, Long term stays allowed, H...",65.0,1,5.00
10,Rental unit in New York · ★4.52 · Studio · 1 b...,Hell's Kitchen,Manhattan,40.76724,-73.98664,Entire home/apt,"[Long term stays allowed, Paid washer – In bui...",175.0,58,4.52
11,Rental unit in New York · ★4.39 · 1 bedroom · ...,East Village,Manhattan,40.72296,-73.98383,Private room,"[Long term stays allowed, Cooking basics, Micr...",70.0,315,4.39


In [14]:
# Group by neighborhood, find average price
avg_price_nbhd_manhattan = manhattan_listings.groupby(["neighbourhood_cleansed"])["price"].mean()

avg_price_nbhd_manhattan = avg_price_nbhd_manhattan.round(2).reset_index()

avg_price_nbhd_manhattan["borough"] = "Manhattan"

avg_price_nbhd_manhattan

,neighbourhood_cleansed,price,borough
0,Battery Park City,268.88,Manhattan
1,Chelsea,290.04,Manhattan
2,Chinatown,210.76,Manhattan
3,Civic Center,288.50,Manhattan
4,East Harlem,158.56,Manhattan
5,East Village,228.85,Manhattan
6,Financial District,315.13,Manhattan
7,Flatiron District,484.31,Manhattan
8,Gramercy,213.86,Manhattan
9,Greenwich Village,262.47,Manhattan


In [15]:
#Find room_type freq by neighborhood:

grouped_man = manhattan_listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')

roomtype_nbhd_manhattan = grouped_man.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)

roomtype_nbhd_manhattan.head()

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_cleansed,,,,
Battery Park City,34.0,0.0,9.0,0.0
Chelsea,424.0,2.0,122.0,6.0
Chinatown,128.0,0.0,88.0,1.0
Civic Center,13.0,0.0,5.0,0.0
East Harlem,293.0,0.0,240.0,13.0


## Top 15 Amenities: Manhattan 

In [16]:
amenities = manhattan_listings[["amenities","neighbourhood_cleansed"]]
df_explode = amenities.explode("amenities")
df_explode.value_counts()["amenities" = ]

amenities              neighbourhood_cleansed
Smoke alarm            Harlem                    1319
Kitchen                Harlem                    1282
Wifi                   Harlem                    1267
Essentials             Harlem                    1238
Carbon monoxide alarm  Harlem                    1127
Hangers                Harlem                    1095
Wifi                   Midtown                   1025
Heating                Harlem                    1016
Hot water              Harlem                    1013
Kitchen                Upper East Side           1006
Hair dryer             Harlem                    1003
Smoke alarm            Midtown                    998
Iron                   Harlem                     996
Wifi                   Upper East Side            995
Dishes and silverware  Harlem                     992
Name: count, dtype: int64

### Bronx Aggregates

In [17]:
bronx_listings = reduced_listings[listings_df["neighbourhood_group_cleansed"] == "Bronx"]
bronx_listings.head()

,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,amenities,price,number_of_reviews,review_scores_rating
98,Rental unit in Bronx · ★4.93 · 1 bedroom · 1 b...,Eastchester,Bronx,40.88100,-73.83511,Entire home/apt,"[Long term stays allowed, Hair conditioner co...",114.0,57,4.93
114,Rental unit in Bronx · ★4.56 · 1 bedroom · 1 b...,Kingsbridge,Bronx,40.87069,-73.90113,Entire home/apt,"[Long term stays allowed, Cooking basics, Pets...",90.0,16,4.56
244,Home in Bronx · ★4.47 · 1 bedroom · 1 bed · 1 ...,University Heights,Bronx,40.85981,-73.90630,Private room,"[Hair dryer, Air conditioning, Iron, Heating, ...",45.0,164,4.47
271,Guest suite in Bronx · ★4.68 · 1 bedroom · 2 b...,Allerton,Bronx,40.86502,-73.85496,Entire home/apt,"[Outdoor dining area, Long term stays allowed,...",175.0,346,4.68
332,Guest suite in Riverdale · ★4.74 · 2 bedrooms...,Spuyten Duyvil,Bronx,40.88095,-73.91701,Entire home/apt,"[Outdoor dining area, Long term stays allowed,...",151.0,146,4.74


In [18]:
# Group by neighborhood, find average price
avg_price_nbhd_bronx = bronx_listings.groupby(["neighbourhood_cleansed"])["price"].mean()

avg_price_nbhd_bronx = avg_price_nbhd_bronx.round(2).reset_index()

avg_price_nbhd_bronx["borough"] = "Bronx"

avg_price_nbhd_bronx

,neighbourhood_cleansed,price,borough
0,Allerton,101.76,Bronx
1,Baychester,99.20,Bronx
2,Belmont,104.35,Bronx
3,Bronxdale,72.88,Bronx
4,Castle Hill,123.20,Bronx
5,City Island,182.31,Bronx
6,Claremont Village,111.94,Bronx
7,Clason Point,105.72,Bronx
8,Co-op City,78.60,Bronx
9,Concourse,98.29,Bronx


In [19]:
#Find room_type freq by neighborhood:
grouped_bronx = bronx_listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')

roomtype_nbhd_bronx = grouped_bronx.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)

roomtype_nbhd_bronx.head()

room_type,Entire home/apt,Private room,Shared room
neighbourhood_cleansed,,,
Allerton,22.0,15.0,0.0
Baychester,10.0,13.0,2.0
Belmont,11.0,15.0,0.0
Bronxdale,6.0,9.0,1.0
Castle Hill,4.0,1.0,0.0


## Top 15 Amenities: Bronx 

### Queens Aggregates

In [20]:
queens_listings = reduced_listings[listings_df["neighbourhood_group_cleansed"] == "Queens"]
queens_listings.head()

,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,amenities,price,number_of_reviews,review_scores_rating
12,Townhouse in Queens · ★4.91 · 1 bedroom · 1 be...,Long Island City,Queens,40.74757,-73.94571,Private room,"[Private backyard – Fully fenced, Coffee, Gard...",165.0,385,4.91
25,Condo in Queens · ★5.0 · 1 bedroom · 1 bed · 1...,Woodside,Queens,40.74884,-73.90536,Private room,"[Air conditioning, Heating, Washer, Dryer, Ele...",65.0,30,5.00
55,Rental unit in Long Island City · ★4.93 · 1 be...,Sunnyside,Queens,40.74523,-73.92318,Entire home/apt,"[Long term stays allowed, Cooking basics, GE s...",78.0,32,4.93
65,Townhouse in Queens · ★4.42 · 5 bedrooms · 10 ...,Ridgewood,Queens,40.70309,-73.89963,Entire home/apt,"[Hair dryer, Air conditioning, Long term stays...",500.0,13,4.42
70,Guesthouse in Queens · ★4.34 · 1 bedroom · 1 b...,Middle Village,Queens,40.71567,-73.87842,Entire home/apt,"[Air conditioning, Heating, Free parking on pr...",130.0,33,4.34


In [21]:
# Group by neighborhood, find average price
avg_price_nbhd_queens = queens_listings.groupby(["neighbourhood_cleansed"])["price"].mean()

avg_price_nbhd_queens = avg_price_nbhd_queens.round(2).reset_index()

avg_price_nbhd_queens["borough"] = "Queens"


avg_price_nbhd_queens

,neighbourhood_cleansed,price,borough
0,Arverne,195.36,Queens
1,Astoria,112.43,Queens
2,Bay Terrace,152.20,Queens
3,Bayside,142.55,Queens
4,Bayswater,94.23,Queens
5,Belle Harbor,223.71,Queens
6,Bellerose,128.27,Queens
7,Breezy Point,150.00,Queens
8,Briarwood,121.84,Queens
9,Cambria Heights,126.53,Queens


In [22]:
#Find room_type freq by neighborhood:
grouped_qu = queens_listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')

roomtype_nbhd_queens = grouped_qu.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)

roomtype_nbhd_queens.head()

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_cleansed,,,,
Arverne,52.0,0.0,28.0,0.0
Astoria,199.0,0.0,304.0,8.0
Bay Terrace,3.0,0.0,2.0,0.0
Bayside,27.0,0.0,12.0,5.0
Bayswater,5.0,0.0,8.0,0.0


### Brooklyn Aggregates

In [23]:
brooklyn_listings = reduced_listings[listings_df["neighbourhood_group_cleansed"] == "Brooklyn"]
brooklyn_listings.head()

,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,amenities,price,number_of_reviews,review_scores_rating
1,Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...,Bedford-Stuyvesant,Brooklyn,40.685350,-73.955120,Private room,"[Air conditioning, Wifi, Kitchen, Heating]",66.0,50,4.52
2,Rental unit in Brooklyn · ★4.58 · 2 bedrooms ·...,Williamsburg,Brooklyn,40.709350,-73.953420,Entire home/apt,"[Cooking basics, Microwave, Extra pillows and ...",81.0,191,4.58
5,Home in Brooklyn · ★4.88 · 1 bedroom · 2 beds ...,Fort Greene,Brooklyn,40.691940,-73.973890,Private room,"[Outdoor dining area, Long term stays allowed,...",205.0,355,4.88
6,Loft in Brooklyn · ★4.91 · 1 bedroom · 1 bed ·...,Williamsburg,Brooklyn,40.712480,-73.958810,Private room,"[Air conditioning, Iron, Heating, Essentials, ...",95.0,13,4.91
7,Place to stay in Brooklyn · ★4.90 · 1 bed · 1 ...,Williamsburg,Brooklyn,40.718807,-73.956177,Entire home/apt,"[Long term stays allowed, Stainless steel sing...",350.0,12,4.90


In [24]:
# Group by neighborhood, find average price
avg_price_nbhd_brooklyn = brooklyn_listings.groupby(["neighbourhood_cleansed"])["price"].mean()

avg_price_nbhd_brooklyn = avg_price_nbhd_brooklyn.round(2).reset_index()

avg_price_nbhd_brooklyn["borough"] = "Brooklyn"

avg_price_nbhd_brooklyn

,neighbourhood_cleansed,price,borough
0,Bath Beach,167.14,Brooklyn
1,Bay Ridge,107.56,Brooklyn
2,Bedford-Stuyvesant,136.80,Brooklyn
3,Bensonhurst,103.93,Brooklyn
4,Bergen Beach,240.22,Brooklyn
5,Boerum Hill,230.23,Brooklyn
6,Borough Park,86.62,Brooklyn
7,Brighton Beach,152.70,Brooklyn
8,Brooklyn Heights,207.34,Brooklyn
9,Brownsville,137.92,Brooklyn


In [25]:
#Find room_type freq by neighborhood:
grouped_bro = brooklyn_listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')

roomtype_nbhd_brooklyn = grouped_qu.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)

roomtype_nbhd_brooklyn.head()

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_cleansed,,,,
Arverne,52.0,0.0,28.0,0.0
Astoria,199.0,0.0,304.0,8.0
Bay Terrace,3.0,0.0,2.0,0.0
Bayside,27.0,0.0,12.0,5.0
Bayswater,5.0,0.0,8.0,0.0


### Staten Island Aggregates

In [31]:
staten_island_listings = reduced_listings[listings_df["neighbourhood_group_cleansed"] == "Staten Island"]
staten_island_listings.tail()

,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,amenities,price,number_of_reviews,review_scores_rating
27232,Home in Staten Island · ★4.82 · 2 bedrooms · 4...,Castleton Corners,Staten Island,40.625570,-74.130790,Private room,"[Outdoor dining area, Long term stays allowed,...",282.0,10,4.82
27236,Rental unit in Staten Island · 1 bedroom · 1 b...,Arrochar,Staten Island,40.597949,-74.081085,Entire home/apt,"[Fire extinguisher, Air conditioning, Outdoor ...",100.0,1,5.00
27424,Home in Staten Island · ★4.67 · 1 bedroom · 1 ...,Mariners Harbor,Staten Island,40.631306,-74.159724,Private room,"[Long term stays allowed, Backyard, Cooking ba...",95.0,6,4.67
27482,Home in Staten Island · 2 bedrooms · 2 beds · ...,"Bay Terrace, Staten Island",Staten Island,40.563230,-74.144842,Entire home/apt,"[Long term stays allowed, Cooking basics, Pets...",171.0,1,5.00
27642,Rental unit in Staten Island · ★4.33 · 2 bedro...,St. George,Staten Island,40.642550,-74.083744,Entire home/apt,"[Air conditioning, Lockbox, Wifi, Self check-i...",95.0,6,4.33


In [27]:
# Group by neighborhood, find average price
avg_price_nbhd_staten_island = staten_island_listings.groupby(["neighbourhood_cleansed"])["price"].mean()

avg_price_nbhd_staten_island = avg_price_nbhd_staten_island.round(2).reset_index()

avg_price_nbhd_staten_island["borough"] = "Staten Island"

avg_price_nbhd_staten_island

,neighbourhood_cleansed,price,borough
0,Arden Heights,140.50,Staten Island
1,Arrochar,128.09,Staten Island
2,"Bay Terrace, Staten Island",171.00,Staten Island
3,Bull's Head,118.86,Staten Island
4,Castleton Corners,160.80,Staten Island
5,"Chelsea, Staten Island",70.00,Staten Island
6,Clifton,139.50,Staten Island
7,Concord,108.71,Staten Island
8,Dongan Hills,142.50,Staten Island
9,Eltingville,131.67,Staten Island


In [28]:
#Find room_type freq by neighborhood:
grouped_sti = staten_island_listings.groupby(['neighbourhood_cleansed', 'room_type']).size().reset_index(name='count')

roomtype_nbhd_statenisland = grouped_sti.pivot(index='neighbourhood_cleansed', columns='room_type', values='count').fillna(0)

roomtype_nbhd_statenisland.head()

room_type,Entire home/apt,Private room,Shared room
neighbourhood_cleansed,,,
Arden Heights,4.0,0.0,0.0
Arrochar,8.0,3.0,0.0
"Bay Terrace, Staten Island",1.0,0.0,0.0
Bull's Head,3.0,4.0,0.0
Castleton Corners,4.0,1.0,0.0


## Combine all aggregates and export files

In [29]:
# Concatenate all average price by neighborhood
avg_price_nbhd_all = pd.concat([avg_price_nbhd_bronx,avg_price_nbhd_brooklyn,avg_price_nbhd_manhattan,avg_price_nbhd_queens,avg_price_nbhd_staten_island], axis=0)

# Reset index to save neighborhood name
avg_price_nbhd_all = avg_price_nbhd_all.rename(columns={"neighbourhood_cleansed": "neighbourhood"})

avg_price_nbhd_all.to_json("data/avg_price_nbhd_all.json", orient="records")

avg_price_nbhd_all

,neighbourhood,price,borough
0,Allerton,101.76,Bronx
1,Baychester,99.20,Bronx
2,Belmont,104.35,Bronx
3,Bronxdale,72.88,Bronx
4,Castle Hill,123.20,Bronx
...,...,...,...
37,Tottenville,282.50,Staten Island
38,West Brighton,75.90,Staten Island
39,Westerleigh,153.00,Staten Island
40,Willowbrook,175.00,Staten Island
